In [255]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, precision_score, recall_score

In [265]:
training_data= pd.read_csv("./instance_training_datasets/2000_samples_desriptive_plus_target_training.csv")
test_data = pd.read_csv("./instance_training_datasets/2000_samples_desriptive_plus_target_test.csv")

In [268]:
### Model 1:
y_training_model1_ = training_data.target_task.values
X_training_model1_ = training_data.drop(['dataset_python', 'indeks_dataset_python', 'cluster', 'cell', 'target_task'], axis=1).values


In [269]:
y_test = test_data.target_task.values
X_test = test_data.drop(['dataset_python', 'indeks_dataset_python', 'cluster', 'cell', 'target_task'], axis=1).values

In [270]:
training_data

,"values__cwt_coefficients__coeff_5__w_10__widths_(2, 5, 10, 20)",values__symmetry_looking__r_0.6000000000000001,values__fft_coefficient__attr_.real.__coeff_5,values__fft_coefficient__attr_.real.__coeff_0,"values__cwt_coefficients__coeff_14__w_2__widths_(2, 5, 10, 20)",values__variance,values__augmented_dickey_fuller__attr_.usedlag.__autolag_.AIC.,values__agg_linear_trend__attr_.rvalue.__chunk_len_5__f_agg_.var.,values__change_quantiles__f_agg_.var.__isabs_True__qh_0.6__ql_0.2,values__symmetry_looking__r_0.8,...,values__symmetry_looking__r_0.55,values__change_quantiles__f_agg_.mean.__isabs_True__qh_0.6__ql_0.2,values__energy_ratio_by_chunks__num_segments_10__segment_focus_4,"values__cwt_coefficients__coeff_0__w_20__widths_(2, 5, 10, 20)","values__cwt_coefficients__coeff_13__w_5__widths_(2, 5, 10, 20)",dataset_python,indeks_dataset_python,cluster,cell,target_task
0,0.216499,1.0,0.567495,3.000000e-08,0.000007,0.989583,3.0,0.462459,0.000000,1.0,...,1.0,0.000000,0.003360,0.093338,0.030488,ElectricDevices,12303,0,58,1
1,0.623826,1.0,-4.069523,1.000000e-07,0.088592,0.989583,10.0,0.577561,0.000000,1.0,...,1.0,0.000000,0.004828,0.381810,-0.053849,ElectricDevices,9612,0,58,0
2,0.431916,1.0,-12.537585,-8.000000e-08,0.014641,0.989583,4.0,0.334667,0.000000,1.0,...,1.0,0.000000,0.001891,0.426496,0.109362,ElectricDevices,2019,0,58,1
3,0.368806,1.0,-4.712266,-5.050000e-08,0.000011,0.996825,16.0,0.419624,0.001876,1.0,...,1.0,0.024038,0.006044,-0.139373,0.052863,UWaveGestureLibraryX,504,0,71,0
4,1.762887,1.0,39.360708,6.860000e-08,0.002757,0.997143,15.0,0.303054,0.007418,1.0,...,1.0,0.055584,0.009868,1.481696,-1.706390,FaceFour,106,0,71,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9695,0.018432,1.0,-4.034944,7.970000e-08,1.489261,0.996094,16.0,-0.418311,0.000089,1.0,...,1.0,0.011284,0.010870,1.856679,5.030920,InsectWingbeatSound,1999,4,867,1
9696,6.594296,1.0,-5.963802,-1.800000e-07,0.475522,0.989899,12.0,-0.566424,0.000227,1.0,...,1.0,0.023001,0.016820,5.536986,2.340359,MedicalImages,531,4,868,1
9697,-0.137852,1.0,43.741979,1.091000e-07,0.615987,0.996094,15.0,-0.232140,0.000877,1.0,...,1.0,0.033015,0.012877,-2.069322,1.792510,InsectWingbeatSound,1047,4,913,1
9698,-1.348148,1.0,31.662504,3.490000e-08,0.040187,0.996094,15.0,-0.286371,0.000125,1.0,...,1.0,0.013595,0.017754,-4.610963,-0.100814,InsectWingbeatSound,981,4,914,0


In [271]:
cluster_groups_train = training_data.groupby(['cluster']).groups
cluster_groups_test = test_data.groupby(['cluster']).groups

In [272]:
X_training_model1_.shape

(9700, 324)

In [273]:
def build_model(X_training_model1_, y_training_model1_, fold_split=5):
    five_fold = StratifiedKFold(fold_split)
    f1_scores_folds = []
    conf_matrix_folds = []
    model1 = RandomForestClassifier(100, max_depth=None, min_samples_split=2, max_features="log2")
    
#     model1 = LogisticRegression(C=10)
    
    cnt = 0
    for train_indecies, test_indecies in five_fold.split(X_training_model1_,y_training_model1_ ):
        print("Training Fold {}".format(cnt+1))
        X_train_model1 = X_training_model1_[train_indecies, :]
        X_test_model1 = X_training_model1_[test_indecies, :]

        y_train_model1 = y_training_model1_[train_indecies]
        y_test_model1 = y_training_model1_[test_indecies]

        model1.fit(X_train_model1, y_train_model1)
        preds1 = model1.predict(X_test_model1)

        f1_scores_folds.append(f1_score(y_test_model1, preds1))
        conf_matrix_folds.append(confusion_matrix(y_test_model1, preds1))
        cnt+=1
        print("The f1 score is {}".format(f1_score(y_test_model1, preds1).round(3)))
        print("------"*10)
        
    model1.fit(X_training_model1_, y_training_model1_)
    
    return f1_scores_folds, conf_matrix_folds, model1

def predict_test(model, X_test, y_test):
    preds = model.predict(X_test)
    res = f1_score(y_test, preds)
    print("TEST F1 {} ".format(res))
    return preds, res

In [274]:
##### Model 1
model1_f1_scores, model1_confusion_matrix, model1_model  = build_model(X_training_model1_, y_training_model1_, 5)
print("MEAN F1 {} and std {}".format(np.mean(model1_f1_scores).round(3), np.std(model1_f1_scores).round(3)))
model1_preds, model1_res =  predict_test(model1_model, X_test, y_test)

Training Fold 1
The f1 score is 0.162
------------------------------------------------------------
Training Fold 2
The f1 score is 0.364
------------------------------------------------------------
Training Fold 3
The f1 score is 0.039
------------------------------------------------------------
Training Fold 4
The f1 score is 0.221
------------------------------------------------------------
Training Fold 5
The f1 score is 0.403
------------------------------------------------------------
MEAN F1 0.238 and std 0.133
TEST F1 0.2516886531574414 


In [275]:
##### Model 2

model2_f1_scores, model1_confusion_matrix, model2_model  = build_model(X_training_model1_[cluster_groups_train[0]], y_training_model1_[cluster_groups_train[0]], 5)
print("MEAN F1 {} and std {}".format(np.mean(model2_f1_scores).round(3), np.std(model2_f1_scores).round(3)))
model2_preds, model2_res =  predict_test(model2_model, X_test, y_test)

Training Fold 1
The f1 score is 0.281
------------------------------------------------------------
Training Fold 2
The f1 score is 0.301
------------------------------------------------------------
Training Fold 3
The f1 score is 0.182
------------------------------------------------------------
Training Fold 4
The f1 score is 0.054
------------------------------------------------------------
Training Fold 5
The f1 score is 0.288
------------------------------------------------------------
MEAN F1 0.221 and std 0.094
TEST F1 0.1449654305468259 


In [276]:
##### Model 3

model3_f1_scores, model1_confusion_matrix, model3_model  = build_model(X_training_model1_[cluster_groups_train[1]], y_training_model1_[cluster_groups_train[1]], 5)
print("MEAN F1 {} and std {}".format(np.mean(model3_f1_scores).round(3), np.std(model3_f1_scores).round(3)))
model3_preds, model3_res =  predict_test(model3_model, X_test, y_test)

Training Fold 1
The f1 score is 0.21
------------------------------------------------------------
Training Fold 2
The f1 score is 0.087
------------------------------------------------------------
Training Fold 3
The f1 score is 0.295
------------------------------------------------------------
Training Fold 4
The f1 score is 0.071
------------------------------------------------------------
Training Fold 5
The f1 score is 0.177
------------------------------------------------------------
MEAN F1 0.168 and std 0.082
TEST F1 0.0787687703735371 


In [277]:
##### Model 4

model4_f1_scores, model4_confusion_matrix, model4_model  = build_model(X_training_model1_[cluster_groups_train[2]], y_training_model1_[cluster_groups_train[2]], 5)
print("MEAN F1 {} and std {}".format(np.mean(model4_f1_scores).round(3), np.std(model4_f1_scores).round(3)))
model4_preds, model4_res =  predict_test(model4_model, X_test, y_test)

Training Fold 1
The f1 score is 0.2
------------------------------------------------------------
Training Fold 2
The f1 score is 0.333
------------------------------------------------------------
Training Fold 3
The f1 score is 0.261
------------------------------------------------------------
Training Fold 4
The f1 score is 0.4
------------------------------------------------------------
Training Fold 5
The f1 score is 0.521
------------------------------------------------------------
MEAN F1 0.343 and std 0.111
TEST F1 0.3418405761731427 


In [278]:
##### Model 5

model5_f1_scores, model5_confusion_matrix, model5_model  = build_model(X_training_model1_[cluster_groups_train[3]], y_training_model1_[cluster_groups_train[3]], 5)
print("MEAN F1 {} and std {}".format(np.mean(model5_f1_scores).round(3), np.std(model5_f1_scores).round(3)))
model5_preds, model5_res =  predict_test(model5_model, X_test, y_test)

Training Fold 1
The f1 score is 0.022
------------------------------------------------------------
Training Fold 2
The f1 score is 0.304
------------------------------------------------------------
Training Fold 3
The f1 score is 0.154
------------------------------------------------------------
Training Fold 4
The f1 score is 0.196
------------------------------------------------------------
Training Fold 5
The f1 score is 0.2
------------------------------------------------------------
MEAN F1 0.175 and std 0.091
TEST F1 0.16114539843891815 


In [279]:
##### Model 5

model6_f1_scores, model6_confusion_matrix, model6_model  = build_model(X_training_model1_[cluster_groups_train[4]], y_training_model1_[cluster_groups_train[4]], 5)
print("MEAN F1 {} and std {}".format(np.mean(model6_f1_scores).round(3), np.std(model6_f1_scores).round(3)))
model6_preds, model6_res =  predict_test(model6_model, X_test, y_test)

Training Fold 1
The f1 score is 0.167
------------------------------------------------------------
Training Fold 2
The f1 score is 0.146
------------------------------------------------------------
Training Fold 3
The f1 score is 0.277
------------------------------------------------------------
Training Fold 4
The f1 score is 0.545
------------------------------------------------------------
Training Fold 5
The f1 score is 0.149
------------------------------------------------------------
MEAN F1 0.257 and std 0.152
TEST F1 0.19566379771288614 


In [280]:
diff_f1_scores = np.ones(5)*model1_res - np.array([model2_res, model3_res, model4_res, model5_res, model6_res])

In [289]:
output_stats = pd.DataFrame(pd.value_counts(test_data.cluster).iloc[:-1, ])
output_stats.columns = ["support"]
output_stats["abs difference test (F1 score) aggregated"] = diff_f1_scores[output_stats.index]
output_stats["f1_all_clusters"] = np.ones(5)*model1_res
output_stats["f1_individual"] = np.array([model2_res, model3_res, model4_res, model5_res, model6_res])[output_stats.index]

In [290]:
output_stats

,support,abs difference test (F1 score) aggregated,f1_all_clusters,f1_individual
3,66843,0.090543,0.251689,0.161145
0,42767,0.106723,0.251689,0.144965
4,30360,0.056025,0.251689,0.195664
1,22018,0.172920,0.251689,0.078769
2,230,-0.090152,0.251689,0.341841


In [283]:
print("Train cluster 0, f1 grouped by cluster 0 ", f1_score(model2_preds[cluster_groups_test[0]], y_test[cluster_groups_test[0]]))
print("Train cluster 0, f1 grouped by cluster 1 ", f1_score(model2_preds[cluster_groups_test[1]], y_test[cluster_groups_test[1]]))
print("Train cluster 0, f1 grouped by cluster 2 ", f1_score(model2_preds[cluster_groups_test[2]], y_test[cluster_groups_test[2]]))
print("Train cluster 0, f1 grouped by cluster 3 ", f1_score(model2_preds[cluster_groups_test[3]], y_test[cluster_groups_test[3]]))
print("Train cluster 0, f1 grouped by cluster 4 ", f1_score(model2_preds[cluster_groups_test[4]], y_test[cluster_groups_test[4]]))

Train cluster 0, f1 grouped by cluster 0  0.288143340341896
Train cluster 0, f1 grouped by cluster 1  0.09423909423909425
Train cluster 0, f1 grouped by cluster 2  0.07843137254901962
Train cluster 0, f1 grouped by cluster 3  0.0443307757885763
Train cluster 0, f1 grouped by cluster 4  0.16416695733054296


In [284]:
print("Train cluster 1, f1 grouped by cluster 0 ",f1_score(model3_preds[cluster_groups_test[0]], y_test[cluster_groups_test[0]]))
print("Train cluster 1, f1 grouped by cluster 1 ",f1_score(model3_preds[cluster_groups_test[1]], y_test[cluster_groups_test[1]]))
print("Train cluster 1, f1 grouped by cluster 2 ",f1_score(model3_preds[cluster_groups_test[2]], y_test[cluster_groups_test[2]]))
print("Train cluster 1, f1 grouped by cluster 3 ",f1_score(model3_preds[cluster_groups_test[3]], y_test[cluster_groups_test[3]]))
print("Train cluster 1, f1 grouped by cluster 4 ",f1_score(model3_preds[cluster_groups_test[4]], y_test[cluster_groups_test[4]]))


Train cluster 1, f1 grouped by cluster 0  0.04152728111662245
Train cluster 1, f1 grouped by cluster 1  0.2358519424900581
Train cluster 1, f1 grouped by cluster 2  0.2962962962962963
Train cluster 1, f1 grouped by cluster 3  0.0414621639440334
Train cluster 1, f1 grouped by cluster 4  0.054498473103124265


In [285]:
print("Train cluster 2, f1 grouped by cluster 0 ",f1_score(model4_preds[cluster_groups_test[0]], y_test[cluster_groups_test[0]]))
print("Train cluster 2, f1 grouped by cluster 1 ",f1_score(model4_preds[cluster_groups_test[1]], y_test[cluster_groups_test[1]]))
print("Train cluster 2, f1 grouped by cluster 2 ",f1_score(model4_preds[cluster_groups_test[2]], y_test[cluster_groups_test[2]]))
print("Train cluster 2, f1 grouped by cluster 3 ",f1_score(model4_preds[cluster_groups_test[3]], y_test[cluster_groups_test[3]]))
print("Train cluster 2, f1 grouped by cluster 4 ",f1_score(model4_preds[cluster_groups_test[4]], y_test[cluster_groups_test[4]]))

Train cluster 2, f1 grouped by cluster 0  0.3215709376534119
Train cluster 2, f1 grouped by cluster 1  0.37312047521811764
Train cluster 2, f1 grouped by cluster 2  0.5555555555555556
Train cluster 2, f1 grouped by cluster 3  0.3206312112874248
Train cluster 2, f1 grouped by cluster 4  0.39497931821423476


In [286]:
print("Train cluster 3, f1 grouped by cluster 0 ",f1_score(model5_preds[cluster_groups_test[0]], y_test[cluster_groups_test[0]]))
print("Train cluster 3, f1 grouped by cluster 1 ",f1_score(model5_preds[cluster_groups_test[1]], y_test[cluster_groups_test[1]]))
print("Train cluster 3, f1 grouped by cluster 2 ",f1_score(model5_preds[cluster_groups_test[2]], y_test[cluster_groups_test[2]]))
print("Train cluster 3, f1 grouped by cluster 3 ",f1_score(model5_preds[cluster_groups_test[3]], y_test[cluster_groups_test[3]]))
print("Train cluster 3, f1 grouped by cluster 4 ",f1_score(model5_preds[cluster_groups_test[4]], y_test[cluster_groups_test[4]]))

Train cluster 3, f1 grouped by cluster 0  0.11111111111111109
Train cluster 3, f1 grouped by cluster 1  0.2874043145441893
Train cluster 3, f1 grouped by cluster 2  0.3728813559322034
Train cluster 3, f1 grouped by cluster 3  0.1876314682372739
Train cluster 3, f1 grouped by cluster 4  0.03291008033169215


In [287]:
print("Train cluster 4, f1 grouped by cluster 0 ",f1_score(model6_preds[cluster_groups_test[0]], y_test[cluster_groups_test[0]]))
print("Train cluster 4, f1 grouped by cluster 1 ",f1_score(model6_preds[cluster_groups_test[1]], y_test[cluster_groups_test[1]]))
print("Train cluster 4, f1 grouped by cluster 2 ",f1_score(model6_preds[cluster_groups_test[2]], y_test[cluster_groups_test[2]]))
print("Train cluster 4, f1 grouped by cluster 3 ",f1_score(model6_preds[cluster_groups_test[3]], y_test[cluster_groups_test[3]]))
print("Train cluster 4, f1 grouped by cluster 4 ",f1_score(model6_preds[cluster_groups_test[4]], y_test[cluster_groups_test[4]]))

Train cluster 4, f1 grouped by cluster 0  0.2341010533833244
Train cluster 4, f1 grouped by cluster 1  0.06993006993006994
Train cluster 4, f1 grouped by cluster 2  0.3513513513513514
Train cluster 4, f1 grouped by cluster 3  0.024914726382915614
Train cluster 4, f1 grouped by cluster 4  0.37654320987654327


In [288]:
print("Train all clusters, f1 grouped by cluster 0 ",f1_score(model1_preds[cluster_groups_test[0]], y_test[cluster_groups_test[0]]))
print("Train all clusters, f1 grouped by cluster 1 ",f1_score(model2_preds[cluster_groups_test[1]], y_test[cluster_groups_test[1]]))
print("Train all clusters, f1 grouped by cluster 2 ",f1_score(model3_preds[cluster_groups_test[2]], y_test[cluster_groups_test[2]]))
print("Train all clusters, f1 grouped by cluster 3 ",f1_score(model4_preds[cluster_groups_test[3]], y_test[cluster_groups_test[3]]))
print("Train all clusters, f1 grouped by cluster 4 ",f1_score(model5_preds[cluster_groups_test[4]], y_test[cluster_groups_test[4]]))

Train all clusters, f1 grouped by cluster 0  0.2755639097744361
Train all clusters, f1 grouped by cluster 1  0.09423909423909425
Train all clusters, f1 grouped by cluster 2  0.2962962962962963
Train all clusters, f1 grouped by cluster 3  0.3206312112874248
Train all clusters, f1 grouped by cluster 4  0.03291008033169215
